### Imports

In [1]:
import os
import gc
import glob
import json
from collections import defaultdict
import multiprocessing as mp
from pathlib import Path
from types import SimpleNamespace
from typing import Dict, List, Optional, Tuple
import warnings

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import PIL.Image as Image
from sklearn.metrics import fbeta_score
from sklearn.exceptions import UndefinedMetricWarning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm import tqdm

### Set up data

In [2]:
BASE_PREFIX = Path('/kaggle/input/vesuvius-challenge/')
PREFIX = BASE_PREFIX / 'train/1/'
BUFFER = 45  # Buffer size in x and y direction
Z_START = 8 # First slice in the z direction to use
Z_DIM = 24  # Number of slices in the z direction
TRAINING_EPOCHS = 30000
VALIDATION_EPOCHS = 500
LEARNING_RATE = 0.02
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [3]:
from typing import Optional
class SubvolumeDataset(data.Dataset):
    """Dataset containing cubical subvolumes of image stack, with the possibility
    of data augmentation through random flips and rotations.
    """
    def __init__(self,
                 image_stack: torch.Tensor,
                 label: Optional[torch.Tensor],
                 pixels: torch.Tensor,
                 buffer: int,
                 xy_flip_prob: float = 0.0,
                 z_flip_prob: float = 0.0,
                 xy_rot_prob: float = 0.0):
        """Create a new SubvolumeDataset.
        
        Args:
          image_stack: 3D image data, as a tensor of voxel values of shape
            (z_dim, y_dim, x_dim).
          label: ink labels for the image stack. A tensor of shape (y_dim, x_dim).
            For testing data, instead pass label=None.
          pixels: Tensor listing pixels to use as centers of subvolumes, of
            shape (num_pixels, 2). Each row of pixels gives coordinates (y, x) of
            a single subvolume center.
          buffer: radius of each subvolume in the x and y direction. Thus, each
            subvolume has shape (z_dim, 2*buffer+1, 2*buffer+1).
          xy_flip_prob: Probability of reflecting each item in the x and y
            directions (independently).
          z_flip_prob: Probability of reflecting each item in the z direction.
          xy_rot_prob: Probability of rotating item by 90 degrees in the xy
            plane. If this check is met, there's a 50% chance of a clockwise
            rotation and a 50% chance of a counterclockwise rotation.
        """
        super().__init__()
        self.image_stack = image_stack
        self.label = label
        self.pixels = pixels
        self.buffer = buffer
        self.xy_flip_prob = xy_flip_prob
        self.z_flip_prob = z_flip_prob
        self.xy_rot_prob = xy_rot_prob
    
    def __len__(self):
        return len(self.pixels)
    
    def __getitem__(self, index):
        """Get a subvolume from the dataset.
        
        If the dataset was defined without label data, the returned label will
        be -1.
        """
        # Note! torch.flip returns a copy, not a view -- thus, we expect this
        # to be slower than the vanilla SubvolumeDataset.
        # Flipping in numpy and then converting to torch.Tensor doesn't work,
        # since torch.Tensor can't take a numpy array with a negative stride.
        y, x = self.pixels[index]
        subvolume = self.image_stack[
            :, y-self.buffer:y+self.buffer+1, x-self.buffer:x+self.buffer+1
        ].reshape(1, -1, self.buffer*2+1, self.buffer*2+1) # -> [1, z, y, x]
        
        # Perform transforms
        if random.random() < self.xy_flip_prob:
            subvolume = torch.flip(subvolume, (2,))
        if random.random() < self.xy_flip_prob:
            subvolume = torch.flip(subvolume, (3,))
        if random.random() < self.z_flip_prob:
            subvolume = torch.flip(subvolume, (1,))
        if random.random() < self.xy_rot_prob:
            if random.random() < 0.5:
                subvolume = torch.rot90(subvolume, k=1, dims=(2, 3))
            else:
                subvolume = torch.rot90(subvolume, k=3, dims=(2, 3))
        
        if self.label is None:
            inklabel = -1
        else:
            inklabel = self.label[y, x].view(1)
            
        return subvolume, inklabel
    
    def set_probs(self, xy_flip_prob, z_flip_prob, xy_rot_prob):
        """Set probabilities of data augmentation transforms."""
        self.xy_flip_prob = xy_flip_prob
        self.z_flip_prob = z_flip_prob
        self.xy_rot_prob = xy_rot_prob

### Set up model

In [4]:
class InkDetector(torch.nn.Module):
    def __init__(self, out_channels=1):
        super().__init__()

        filters = [16, 32, 64]
        paddings = [1, 1, 1]
        kernel_sizes = [3, 3, 3]
        strides = [2, 2, 2]
        
        layers = []
        in_channels = 1
        for num_filters, padding, kernel_size, stride in zip(filters, paddings, kernel_sizes, strides):
            layers.extend([
                nn.Conv3d(
                    in_channels=in_channels,
                    out_channels=num_filters,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                ),
                nn.ReLU(inplace=True),
                torch.nn.BatchNorm3d(num_features=num_filters)
            ])
            in_channels = num_filters
        layers.append(nn.AdaptiveAvgPool3d(1))
        layers.append(nn.Flatten())

        self.encoder = nn.Sequential(*layers)
        self.decoder = nn.Sequential(
            nn.Linear(in_channels, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, out_channels)
        )

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

In [5]:
TRAIN_RUN = False

In [6]:
if TRAIN_RUN:
    model = InkDetector().to(DEVICE)
    model.load_state_dict(torch.load('/kaggle/input/pretrained-model-for-ink-detection/InkDetector_full_dataset_augmented_pretrained_100000_epochs.pt'))

### Evaluate

In [7]:
test_path = BASE_PREFIX / "test"
test_fragments = [test_path / fragment_name for fragment_name in test_path.iterdir()]
print("All fragments:", test_fragments)

All fragments: [PosixPath('/kaggle/input/vesuvius-challenge/test/b'), PosixPath('/kaggle/input/vesuvius-challenge/test/a')]


In [8]:
import gc
from typing import Union, Optional, Tuple, List
import random

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

def get_rect_dset(
    fragment_path: Union[Path, str],
    z_start: int,
    z_dim: int,
    buffer: int,
    rect: Optional[Tuple[int]] = None,
    shuffle: bool = False,
) -> data.Dataset:
    """Get a dataset consisting of a rectangle from a single fragment.
    
    Args:
      fragment_path: Path to folder containing fragment data (e.g., 'data/train/1')
      z_start: Lowest z-value to use from the fragment. Should be between 0
        and 64. 
      z_dim: Number of z-values to use.
      buffer: Radius of subvolumes in x and y directions. Thus, each item in
        the dataset will be a subvolume of size 2*buffer+1 x 2*buffer+1 x z_dim.
      rect: Rectangle to use from the fragment. Should be a tuple of the form:
          (top_left_corner_x, top_left_corner_y, width, height)
        Use show_labels_with_rects() to double-check the rectangle. If rect is
        None, the whole dataset will be used.
      shuffle: Whether to shuffle the dataset before returning it.
    
    Returns:
      A dataset consisting of subvolumes from the given fragment inside the
      given rectangle.
    """
    # clean input
    fragment_path = Path(fragment_path)
    
    images = [
        np.array(Image.open(filename), dtype=np.float32)/65535.0
        for filename
        in tqdm(
            sorted((fragment_path / "surface_volume").glob("*.tif"))[z_start : z_start + z_dim],
            desc=f"Loading fragment from {fragment_path}"
        )
    ]
        
    # turn images to tensors
    image_stack = torch.stack([torch.from_numpy(image) for image in images], 
                              dim=0)

    # get mask and labels
    mask = np.array(Image.open(fragment_path / "mask.png").convert('1'))
    if os.path.exists(fragment_path / "inklabels.png"):
        label = torch.from_numpy(
            np.array(Image.open(fragment_path / "inklabels.png"))
        ).float()
    else:
        label = None

    # Create a Boolean array of the same shape as the bitmask, initially all True
    not_border = np.zeros(mask.shape, dtype=bool)
    not_border[buffer:mask.shape[0]-buffer, buffer:mask.shape[1]-buffer] = True
    arr_mask = mask * not_border
    if rect is not None:
        inside_rect = np.zeros(mask.shape, dtype=bool) * arr_mask
        # Sets all indexes with inside_rect array to True
        inside_rect[rect[1]:rect[1]+rect[3]+1, rect[0]:rect[0]+rect[2]+1] = True
        # Set the pixels within the inside_rect to False
        pixels = torch.tensor(np.argwhere(inside_rect))
    else:
        pixels = torch.tensor(np.argwhere(arr_mask))
        
    if shuffle:
        perm = torch.randperm(len(pixels))
        pixels = pixels[perm]

    # define dataset
    return SubvolumeDataset(image_stack, label, pixels, buffer)

def predict_on_test_fragments(model: torch.nn.Module,
                              fragment_path: Union[Path, str],
                              z_start: int,
                              z_dim: int,
                              buffer: int, 
                              batch_size: int,
                              decision_boundary: float = 0.4) -> np.ndarray:
    """Generate predictions on a fragment.
    
    Args:
      model: Model to use for prediction.
      fragment_path: Path to fragment (e.g. '/data/test/a').
      z_start: z-value of lowest fragment layer to use.
      z_dim: Number of layers to use.
      buffer: Radius of subvolumes in x and y direction. Thus, each subvolume
        is an array of shape (z_dim, 2*buffer+1, 2*buffer+1).
      batch_size: Number of subvolumes in one batch.
      decision_boundary: Decision boundary for predictions. If the model
        returns a probability above this number for a given pixel, we classify
        that pixel as containing ink.
    
    Returns:
      Predictions on the fragment, as a boolean NumPy array.
    """
    fragment_path = Path(fragment_path)
    model.eval()
    outputs = []
    test_dset = get_rect_dset(fragment_path, z_start=z_start,
                              z_dim=z_dim, buffer=buffer)
    test_loader = data.DataLoader(test_dset, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        for i, (subvolumes, _) in enumerate(tqdm(test_loader, desc=f"Predicting on fragment {fragment_path}")):
            output = model(subvolumes.to(DEVICE)).view(-1).sigmoid().cpu().numpy()
            outputs.append(output)
    image_shape = test_dset.image_stack[0].shape

    pred_image = np.zeros(image_shape, dtype=np.uint8)
    outputs = np.concatenate(outputs)
    for (y, x, _), prob in zip(test_dset.pixels[:outputs.shape[0]], outputs):
        pred_image[y, x] = prob > decision_boundary
    pred_images.append(pred_image)

    print("Finished with fragment", test_fragment)
    return pred_image
                                            
    
def rle(output: np.ndarray) -> str:
    """Turn a NumPy array of booleans to a run-length encoded string."""
    flat_img = np.where(output > 0.4, 1, 0).astype(np.uint8)
    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix
    return " ".join(map(str, sum(zip(starts_ix, lengths), ())))

                                                
def make_csv(pred_images: List[np.ndarray], save_path: Union[Path, str, None]):
    """Save a list of predicted images as a run-length encoded CSV file.
    
    Args:
      pred_images: List of two NumPy arrays of predictions (which we assume
        correspond to test fragments a and b respectively).
      save_path: Path to save CSV file (e.g. '/kaggle/working/submission.csv').
        If None, the file will not be saved.
    Returns:
      A DataFrame containing run-length encodings of the two fragments.
    """
    submission = defaultdict(list)
    for fragment_id, fragment_name in enumerate(['a', 'b']):
        submission["Id"].append(fragment_name)
        submission["Predicted"].append(rle(pred_images[fragment_id]))
    dataframe = pd.DataFrame.from_dict(submission)
    dataframe.to_csv(save_path, index=False)
    return dataframe

In [9]:
if TRAIN_RUN:
    pred_images = []
    model.eval()
    for test_fragment in test_fragments:
        pred_images.append(predict_on_test_fragments(model, test_fragment, Z_START, Z_DIM, BUFFER, BATCH_SIZE,
                                                     decision_boundary=0.4))
    make_csv(pred_images, '/kaggle/working/submission.csv')
    plt.imshow(pred_images[0], cmap='gray')
else:
    import shutil
    shutil.copy(src='/kaggle/input/submissioncsv/submission_InkDetector_full_dataset_augmented_pretrained_100000_epochs.csv',
                dst='/kaggle/working/submission.csv')

### Submission